In [ ]:
from typing import Dict
from typing import Generator
from typing import Union

import pandas as pd

from evidently import ColumnType
from evidently.future.datasets import DataDefinition

from evidently.future.datasets import DatasetColumn

from evidently.future.datasets import Descriptor
from evidently.future.metric_types import Metric
from evidently.future.datasets import Dataset
from evidently.future.metric_types import MetricResult
from evidently.future.metric_types import MetricTestResult
from evidently.future.metric_types import SingleValue
from evidently.future.metric_types import SingleValueTest
from evidently.future.metric_types import MetricCalculation
from evidently.future.metric_types import MetricId
from evidently.future.metric_types import SingleValueMetric
from evidently.future.metric_types import TResult
from evidently.future.preset_types import PresetResult

In [ ]:
from typing import Optional


class TextLengthScorer(Descriptor):
    def __init__(self, column_name: str, alias: Optional[str] = None):
        super().__init__(alias or f"{column_name}: Text Length")
        self._column_name = column_name

    def generate_data(self, dataset: "Dataset") -> Union[DatasetColumn, Dict[str, DatasetColumn]]:
        lengths = dataset.column(self._column_name).data.apply(len)
        return DatasetColumn(type=ColumnType.Numerical, data=lengths)


In [ ]:
class ToxicityScorer(Descriptor):
    def __init__(self, column_name: str, alias: Optional[str] = None):
        super().__init__(alias or f"{column_name}: Toxicity")
        self._column_name = column_name

    def generate_data(self, dataset: "Dataset") -> Union[DatasetColumn, Dict[str, DatasetColumn]]:
        from evidently.descriptors import ToxicityLLMEval
        from evidently.options.base import Options

        feature = ToxicityLLMEval().feature(self._column_name)
        data = feature.generate_features(dataset.as_dataframe(), None, Options())
        return {
            col: DatasetColumn(type=feature.get_type(f"{feature.get_fingerprint()}.{col}"), data=data[col])
            for col in data.columns
        }

In [ ]:
def my_scorer(data: DatasetColumn) -> DatasetColumn:
    return DatasetColumn(type=data.type, data=data.data)

def my_scorer2(dataset: Dataset) -> Union[DatasetColumn, Dict[str, DatasetColumn]]:
    return {"c1": dataset.column("column_1"), "c2": dataset.column("column_2")}

In [ ]:
from evidently.future.datasets import ColumnInfo
from evidently.future.descriptors import CustomColumnDescriptor
from evidently.future.descriptors import CustomDescriptor
from evidently.future.descriptors import TextLength

data = pd.DataFrame(data={"column_1": [1, 2, 3, 4, -1, 5], "column_2": ["a", "aa", "aaaa", "aaaaaaa", "a", "aa"]})

dataset = Dataset.from_pandas(
    data,
    data_definition=DataDefinition(
        numerical_columns=["column_1"],
        categorical_columns=["column_2"],
    ),
    descriptors=[
        TextLength("column_2", alias="column 2 length"),
        TextLength("column_2", alias="column 2 length"),
        # ToxicityScorer("column_2"),
        CustomColumnDescriptor("column_2", my_scorer, alias="column 2 custom function"),
        CustomDescriptor(my_scorer2, alias="global custom function"),
    ],
)

dataset.as_dataframe()

In [ ]:
from evidently.future.tests import Reference, eq
from evidently.future.metric_types import BoundTest
from evidently.future.report import Context
from typing import Optional
from typing import List
from plotly.express import line


class MyMaxMetric(SingleValueMetric):
    column: str

    def _default_tests(self) -> List[BoundTest]:
        return [eq(0).bind_single(self.get_fingerprint())]

    def _default_tests_with_reference(self) -> List[BoundTest]:
        return [eq(Reference(relative=0.1)).bind_single(self.get_fingerprint())]

# implementation
class MaxMetricImplementation(MetricCalculation[SingleValue, MyMaxMetric]):
    def calculate(self, context: Context, current_data: Dataset, reference_data: Optional[Dataset]) -> SingleValue:
        x = current_data.column(self.metric.column).data
        value = x.max()
        result = SingleValue(value=value)
        figure = line(x)
        figure.add_hrect(6, 10)
        #result.set_widget([plotly_figure(title=self.display_name(), figure=figure)])
        return result

    def display_name(self) -> str:
        return f"Max value for {self.metric.column}"


from evidently.future.report import Context
context = Context(None)

context.init_dataset(dataset, None)

result = MyMaxMetric(column="column_1").to_calculation().call(context)
result

In [ ]:
from evidently.future.metrics.group_by import GroupBy
from evidently.future.metric_types import render_results

context = Context(None)

context.init_dataset(dataset, None)

metrics = GroupBy(MyMaxMetric(column="column 2 length"), "column_1").generate_metrics(context)

results = [metric.call(context) for metric in metrics]

render_results(results)

In [ ]:
results[0][0].value

In [ ]:
from evidently.future.preset_types import MetricPreset
from evidently.future.metrics import MinValue
from evidently.future.metrics import MaxValue

class ColumnSummary(MetricPreset):
    def __init__(self, column: str):
        self._column = column

    def metrics(self) -> List[Metric]:
        return [
            MinValue(column=self._column),
            MaxValue(column=self._column),
        ]
    
    def calculate(self, metric_results: Dict[MetricId, MetricResult]) -> PresetResult:
        return PresetResult(widget=[
            *metric_results[MinValue(column=self._column).get_metric_id()].widget,
            *metric_results[MaxValue(column=self._column).get_metric_id()].widget,
        ])


In [ ]:
from evidently.future.report import Report
from evidently.future.tests import lte

report = Report([
    MyMaxMetric(column="column 2 length", tests=[lte(100), lte(3)]),
    MyMaxMetric(column="column_1", tests=[lte(100)]),
    MyMaxMetric(column="global custom function.c1", tests=[lte(100)]),
    ColumnSummary("column_1"),
    GroupBy(MyMaxMetric(column="column 2 length"), "column_1"),
])
snapshot = report.run(dataset, None)
snapshot